##Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from google.colab import drive
warnings.filterwarnings('ignore')

In [ ]:
!pip install librosa numpy praat-parselmouth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 89.5 MB/s eta 0:00:00


In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import librosa
import numpy as np
import parselmouth
from parselmouth.praat import call

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/IPD/code/audio/augmented_audio_paths_modified.csv")
df.head()

,Unnamed: 0,index,text,hate,audio,augment
0,0,48.0,@odysseuslahori @IqShoaib Hang till death all ...,1.0,/content/drive/MyDrive/IPD/code/audio/claude_t...,/content/drive/MyDrive/IPD/code/audio/augmente...
1,1,84.0,"""where they ultimately need to be"" is back in ...",1.0,/content/drive/MyDrive/IPD/code/audio/claude_t...,/content/drive/MyDrive/IPD/code/audio/augmente...
2,2,126.0,"""I'm trying to take you out"" to ""you don't loo...",1.0,/content/drive/MyDrive/IPD/code/audio/claude_t...,/content/drive/MyDrive/IPD/code/audio/augmente...
3,3,298.0,Facts . That's why I cut these hoes off,1.0,/content/drive/MyDrive/IPD/code/audio/claude_t...,/content/drive/MyDrive/IPD/code/audio/augmente...
4,4,454.0,"""I'm a whore! Yay! Glorify being a whore! This...",1.0,/content/drive/MyDrive/IPD/code/audio/claude_t...,/content/drive/MyDrive/IPD/code/audio/augmente...


##Feature Extraction

In [ ]:
def extract_audio_features(audio_path):
    y, sr = librosa.load(audio_path, sr=None)  # y is the audio time series, sr is the sample rate

    features = {}

    # 1. Pitch (Fundamental Frequency)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitches = pitches[pitches > 0]  # Filter out zeros
    features['pitch_mean'] = np.mean(pitches) if len(pitches) > 0 else 0
    features['pitch_std'] = np.std(pitches) if len(pitches) > 0 else 0

    # 2. Spectral Centroid
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    features['spectral_centroid_mean'] = np.mean(spectral_centroid)
    features['spectral_centroid_std'] = np.std(spectral_centroid)

    # 3. Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(y)
    features['zcr_mean'] = np.mean(zcr)
    features['zcr_std'] = np.std(zcr)

    # 4. Root Mean Square Energy (RMS)
    rms = librosa.feature.rms(y=y)
    features['rms_mean'] = np.mean(rms)
    features['rms_std'] = np.std(rms)

    # 5. Spectral Rolloff
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr, roll_percent=0.85)
    features['spectral_rolloff_mean'] = np.mean(spectral_rolloff)
    features['spectral_rolloff_std'] = np.std(spectral_rolloff)

    # 6. Speaking Rate
    hop_length = 512  # Default hop length in librosa
    duration = librosa.get_duration(y=y, sr=sr)
    voiced_frames = librosa.effects.split(y, top_db=20)  # Split based on silence
    speaking_rate = len(voiced_frames) / duration if duration > 0 else 0
    features['speaking_rate'] = speaking_rate

    # 7. Harmonic-to-Noise Ratio
    try:
        sound = parselmouth.Sound(audio_path)
        pitch = call(sound, "To Pitch", 0.0, 75, 600)
        harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
        hnr_values = []
        for time in pitch.ts():
            harmonicity_value = call(harmonicity, "Get value at time", time, "Linear")
            if not np.isnan(harmonicity_value):
                hnr_values.append(harmonicity_value)

        hnr_mean = sum(hnr_values) / len(hnr_values) if len(hnr_values) > 0 else 0
        hnr_std = np.std(hnr_values) if len(hnr_values) > 1 else 0
        features["hnr_mean"] = hnr_mean
        features["hnr_std"] = hnr_std

    except Exception as e:
        print(f"Error calculating HNR: {e}")
        features["hnr_mean"] = 0
        features["hnr_std"] = 0
    return features

In [ ]:
extract_audio_features(df['audio'][0])

{'pitch_mean': 1458.9489,
 'pitch_std': 1122.4113,
 'spectral_centroid_mean': 2528.35677489655,
 'spectral_centroid_std': 2069.900019920371,
 'zcr_mean': 0.12556875218226257,
 'zcr_std': 0.16939099511939446,
 'rms_mean': 0.08379578,
 'rms_std': 0.06489106,
 'spectral_rolloff_mean': 4404.384165502794,
 'spectral_rolloff_std': 3283.588469842132,
 'speaking_rate': 1.310272536687631,
 'hnr_mean': -59.82541865716342,
 'hnr_std': 100.61475483803599}

In [ ]:
columns = ['index','pitch_mean', 'pitch_std', 'spectral_centroid_mean', 'spectral_centroid_std',
                                    'zcr_mean', 'zcr_std', 'rms_mean', 'rms_std', 'spectral_rolloff_mean',
                                    'spectral_rolloff_std', 'speaking_rate', 'hnr_mean', 'hnr_std','hate']
feature_df = pd.DataFrame(columns=columns)
for index, row in df.iterrows():
    id = row['index'] if not np.isnan(row['index']) else df['index'][index-1000] + 0.1
    audio_path = row['augment']
    features = extract_audio_features(audio_path)
    features['index'] = id
    features['hate'] = row['hate']
    feature_df.loc[len(feature_df)] = features

In [ ]:
feature_df.to_csv("/content/drive/MyDrive/IPD/code/audio/feature_extracted.csv")

In [ ]:
feature_df = pd.read_csv("/content/drive/MyDrive/IPD/code/audio/feature_extracted.csv")
feature_df.drop(columns=["Unnamed: 0"],inplace=True)
feature_df.head()

,index,pitch_mean,pitch_std,spectral_centroid_mean,spectral_centroid_std,zcr_mean,zcr_std,rms_mean,rms_std,spectral_rolloff_mean,spectral_rolloff_std,speaking_rate,hnr_mean,hnr_std,hate
0,48.0,1947.1307,1143.7485,4378.746982,1410.851612,0.257575,0.187818,0.088702,0.060642,8825.233502,1323.673214,1.441300,-62.506859,98.441387,1.0
1,84.0,1591.7039,1099.0455,3034.581428,1378.520063,0.138716,0.139052,0.068526,0.042545,6412.378772,2409.543056,0.810636,-36.677607,88.585121,1.0
2,126.0,1188.5610,950.0717,1893.726573,1256.041650,0.072548,0.074697,0.037494,0.026806,3655.643439,2759.555598,0.971251,-61.680579,97.244589,1.0
3,298.0,1519.5168,994.9424,3255.370323,2274.918026,0.166019,0.177654,0.066600,0.068555,5721.109480,3701.307650,1.893939,-107.198889,106.638173,1.0
4,454.0,1457.0230,1060.3801,2268.857176,1897.603370,0.117746,0.140863,0.076510,0.058947,4075.818650,3445.456428,2.245509,-77.738639,107.043610,1.0


In [ ]:
feature_df.shape

(2000, 15)

##Model Creation

In [ ]:
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model
import torchaudio
import numpy as np

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def extract_embeddings(audio_path):
    waveform, sample_rate = librosa.load(audio_path, sr=None)  # sr=None keeps the original sampling rate

    # Resample the audio to 16000 Hz if it's not already at that rate
    if sample_rate != 16000:
        waveform = librosa.resample(waveform, orig_sr=sample_rate, target_sr=16000)
        sample_rate = 16000

    # Preprocess the audio to the required format
    inputs = processor(waveform, return_tensors="pt", sampling_rate=sample_rate)

    # Forward pass through the model
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract embeddings (hidden states from the last layer)
    embeddings = outputs.last_hidden_state

    # Convert to numpy array if needed
    embeddings_np = embeddings.squeeze().numpy()

    # Optionally, you can aggregate (e.g., take mean) over time steps
    embedding_mean = np.mean(embeddings_np, axis=0)

    return embedding_mean

In [ ]:
feature_df.iloc[0]

,0
index,48.000000
pitch_mean,1947.130700
pitch_std,1143.748500
spectral_centroid_mean,4378.746982
spectral_centroid_std,1410.851612
zcr_mean,0.257575
zcr_std,0.187818
rms_mean,0.088702
rms_std,0.060642
spectral_rolloff_mean,8825.233502


In [ ]:
y = df['hate']
y

,hate
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
1995,0.0
1996,0.0
1997,0.0
1998,0.0


In [ ]:
x = extract_embeddings(df['augment'][0])

In [ ]:
x.shape

(768,)

In [ ]:
np.concatenate([x,feature_df.iloc[0].drop('hate').drop('index').to_numpy()]).shape

(781,)

In [ ]:
X = np.load("/content/drive/MyDrive/IPD/code/audio/744_rows.npy")

In [ ]:
fr_df = feature_df.drop(columns=['index','hate'])
fr_df.iloc[0]

,0
pitch_mean,1947.130700
pitch_std,1143.748500
spectral_centroid_mean,4378.746982
spectral_centroid_std,1410.851612
zcr_mean,0.257575
zcr_std,0.187818
rms_mean,0.088702
rms_std,0.060642
spectral_rolloff_mean,8825.233502
spectral_rolloff_std,1323.673214


In [ ]:
for index, row in df.iterrows():
    if index < 744:
        continue
    print(index)
    x = extract_embeddings(row['augment'])
    X = np.vstack((X,np.concatenate([x,fr_df.iloc[index].to_numpy()])))

744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993


In [ ]:
np.save("/content/drive/MyDrive/IPD/code/audio/wav2vecAndFeature.npy",X)